# Defacing pre-registration - Statistical analysis in R

## Load the simulated or collected manual ratings

In [1]:
#df <- readRDS(("../../data/simulated_normal_ratings.rds"))
df <- read.csv(file = 'IXI_augmented_ratings_df.tsv', sep='\t')
# Drop columns we will not use in the analysis
df <- df[, !names(df) %in% c("dataset", "artifacts", "time_sec", "confidence", "comments", "randomized_id")]
# sort dataframe by subject ID
df <- df[order(df$subject),]
df

,rater_id,rating,defaced,subject
,<chr>,<dbl>,<int>,<int>
49,rater_03,3.45,0,12
100,rater_01,3.40,0,12
134,rater_04,3.50,0,12
192,rater_02,3.65,0,12
787,rater_04,3.90,1,12
1038,rater_02,3.50,1,12
1410,rater_03,2.75,1,12
1659,rater_01,3.40,1,12
180,rater_04,3.45,0,13


In [2]:
library(dplyr)
# When there are two ratings corresponding to one image, keep first rating only, as rm-ANOVA is not able to deal with duplicates
df_nodup <- df[!duplicated(df[c("subject", "defaced", "rater_id")]), ]
nrow(df_nodup)

print(df_nodup)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] 1480

     rater_id rating defaced subject
49   rater_03   3.45       0      12
100  rater_01   3.40       0      12
134  rater_04   3.50       0      12
192  rater_02   3.65       0      12
787  rater_04   3.90       1      12
1038 rater_02   3.50       1      12
1410 rater_03   2.75       1      12
1659 rater_01   3.40       1      12
180  rater_04   3.45       0      13
238  rater_02   3.50       0      13
301  rater_01   2.75       0      13
399  rater_03   2.85       0      13
777  rater_04   3.10       1      13
1028 rater_02   3.00       1      13
1400 rater_03   3.00       1      13
1649 rater_01   2.55       1      13
516  rater_02   2.65       1      14
517  rater_02   2.80       0      14
699  rater_04   3.70       1      14
700  rater_04   3.05       0      14
1216 rater_03   2.60       1      14
1217 rater_03   2.75       0      14
1571 rater_01   3.20       1      14
1572 rater_01   2.50       0      14
812  rater_04   3.45       1      15
821  rater_04   3.60       0      15
1

## Linear mixed-effects models

We will test if including the defacing factor as fixed effect improves the model fit using a likelihood-ratio test.

In [3]:
## Fit model without defaced as fixed effect
library(lme4)
fm0 <- lmer(as.numeric(rating) ~   (rater_id | subject), data=df_nodup)
summary(fm0)
ranef(fm0)

Loading required package: Matrix



Linear mixed model fit by REML ['lmerMod']
Formula: as.numeric(rating) ~ (rater_id | subject)
   Data: df_nodup

REML criterion at convergence: 1726.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.8897 -0.4917  0.0581  0.5473  3.3811 

Random effects:
 Groups   Name             Variance Std.Dev. Corr             
 subject  (Intercept)      0.19606  0.4428                    
          rater_idrater_02 0.11108  0.3333    0.19            
          rater_idrater_03 0.05653  0.2378    0.12  0.61      
          rater_idrater_04 0.21394  0.4625   -0.60 -0.20  0.02
 Residual                  0.09863  0.3140                    
Number of obs: 1480, groups:  subject, 185

Fixed effects:
            Estimate Std. Error t value
(Intercept)   3.2989     0.0284   116.2

$subject
     (Intercept) rater_idrater_02 rater_idrater_03 rater_idrater_04
12   0.068988776     0.0752607672     -0.070347853      0.212571407
13  -0.406356669     0.2094428276      0.075880184      0.335777378
14  -0.415632461    -0.1605983328     -0.115519102      0.425631755
15  -0.448762033    -0.3916062195     -0.226664864      0.570312231
33  -0.017499775    -0.1176924263     -0.015878842      0.133153747
34  -0.145270928    -0.2131070006      0.030931903      0.369542664
39  -0.044135542     0.0659268124      0.058143625      0.348074204
48  -0.371574752     0.0639769987     -0.207888756      0.192694327
49   0.124888636    -0.1594742175     -0.047092261      0.006077424
51   0.157039197    -0.0102955860      0.011763233      0.151744342
52   0.061505173     0.0264107303      0.024116373      0.271256035
56  -0.047621411    -0.1899224999     -0.016201684      0.400894242
57   0.186966984    -0.1227739918      0.057069440     -0.057798340
59  -0.065004032    -0.0241455364      

In [4]:
## Fit model with defaced as fixed effect
library(lme4)
fm1 <- lmer(as.numeric(rating) ~  (rater_id*defaced| subject), data=df_nodup)
summary(fm1)
ranef(fm1)

ERROR: Error: number of observations (=1480) <= number of random effects (=1480) for term (rater_id * defaced | subject); the random-effects parameters and the residual variance (or scale parameter) are probably unidentifiable


In [ ]:
#Likelihood-ratio test
anova(fm0, fm1)

refitting model(s) with ML (instead of REML)



,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fm0,3,2260.959,2276.858,-1127.479,2254.959,NA,NA,NA
fm1,4,2262.552,2283.751,-1127.276,2254.552,0.4071299,1,0.5234299


The p-value for the likelihood-ratio test is significant (3e-8) meaning that defacing influence on the manual ratings is significant.

## Exploratory analysis


### Linear mixed-effects with only poor quality and excluded scans

In [ ]:
library(dplyr)
df_poor <- df %>%
  filter(rating <= 2.45)
nrow(df_poor)

[1] 186

In [ ]:
## Fit model without defaced as fixed effect
library(lme4)
fm0 <- lmer(as.numeric(rating) ~ (rater_id | subject), data=df_poor)
summary(fm0)
ranef(fm0)

Linear mixed model fit by REML ['lmerMod']
Formula: as.numeric(rating) ~ (1 | rater_id)
   Data: df_poor

REML criterion at convergence: 248.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8355 -0.8955  0.2631  0.8696  1.2686 

Random effects:
 Groups   Name        Variance Std.Dev.
 rater_id (Intercept) 0.01091  0.1045  
 Residual             0.21353  0.4621  
Number of obs: 186, groups:  rater_id, 4

Fixed effects:
            Estimate Std. Error t value
(Intercept)  1.94376    0.06344   30.64

$rater_id
         (Intercept)
rater_01  0.10441200
rater_02 -0.07994910
rater_03  0.03468528
rater_04 -0.05914818

with conditional variances for “rater_id” 

In [ ]:
## Fit model with defaced as fixed effect
library(lme4)
fm1 <- lmer(as.numeric(rating) ~ (1 | defaced) + (1 | rater_id), data=df_poor)
summary(fm1)
ranef(fm1)

Linear mixed model fit by REML ['lmerMod']
Formula: as.numeric(rating) ~ (1 | defaced) + (1 | rater_id)
   Data: df_poor

REML criterion at convergence: 247.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8790 -0.9248  0.2975  0.8944  1.3019 

Random effects:
 Groups   Name        Variance Std.Dev.
 rater_id (Intercept) 0.011003 0.10490 
 defaced  (Intercept) 0.001367 0.03697 
 Residual             0.212820 0.46132 
Number of obs: 186, groups:  rater_id, 4; defaced, 2

Fixed effects:
            Estimate Std. Error t value
(Intercept)  1.94505    0.06878   28.28

$rater_id
         (Intercept)
rater_01  0.10584966
rater_02 -0.07973515
rater_03  0.03361305
rater_04 -0.05972756

$defaced
  (Intercept)
0  -0.0159269
1   0.0159269

with conditional variances for “rater_id” “defaced” 

In [ ]:
#Likelihood-ratio test
anova(fm0, fm1)

refitting model(s) with ML (instead of REML)



,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fm0,3,250.2220,259.8993,-122.1110,244.2220,NA,NA,NA
fm1,4,252.2123,265.1153,-122.1062,244.2123,0.009736727,1,0.9213964
